> https://docs.langchain.com/oss/python/langgraph/thinking-in-langgraph

**Build a customer support email agent**

The agent should:
- Read incoming customer emails
- Classify them by urgency and topic
- Search relevant documentation to answer questions
- Draft appropriate responses
- Escalate complex issues to human agents
- Schedule follow-ups when needed

Example scenarios to handle:
1. Simple product question: “How do I reset my password?”
2. Bug report: “The export feature crashes when I select PDF format”
3. Urgent billing issue: “I was charged twice for my subscription!”
4. Feature request: “Can you add dark mode to the mobile app?”
5. Complex technical issue: “Our API integration fails intermittently with 504 errors”

In [1]:
import os
from dotenv import load_dotenv

from langchain_deepseek import ChatDeepSeek

In [2]:
load_dotenv()

True

In [ ]:
model = ChatDeepSeek(model="deepseek-chat")

# Step 1: Map out your workflow as discrete steps

1. Read Email: Extract and parse the email content
2. Classify Intent: Use an LLM to categorize urgency and topic, then route to appropriate action
3. Doc Search: Query your knowledge base for relevant information
4. Bug Track: Create or update issue in tracking system
5. Draft Reply: Generate an appropriate response
6. Human Review: Escalate to human agent for approval or handling
7. Send Reply: Dispatch the email response

# Step 2: Identify what each step needs to do

For each node in your graph, determine what type of operation it represents and what context it needs to work properly.

# Step 3: Design your state

State is the shared memory accessible to all nodes in your agent. Think of it as the notebook your agent uses to keep track of everything it learns and decides as it works through the process.

## What belongs in state?

- Include in State
    - Does it need to persist across steps? If yes, it goes in state.
- Don't Store
    - Can you derive it from other data? If yes, compute it when needed instead of storing it in state.

For our email agent, we need to track:
- The original email and sender info (can’t reconstruct these)
- Classification results (needed by multiple downstream nodes)
- Search results and customer data (expensive to re-fetch)
- The draft response (needs to persist through review)
- Execution metadata (for debugging and recovery)

## Keep state raw, format prompts on-demand

> A key principle: your state should store raw data, not formatted text. Format prompts inside nodes when you need them.

This separation means:
- Different nodes can format the same data differently for their needs
- You can change prompt templates without modifying your state schema
- Debugging is clearer - you see exactly what data each node received
- Your agent can evolve without breaking existing state

In [6]:
from typing import TypedDict, Literal

# Define the structure for email classification
class EmailClassification(TypedDict):
    intent: Literal["question", "bug", "billing", "feature", "complex"]
    urgency: Literal["low", "medium", "high", "critical"]
    topic: str
    summary: str

class EmailAgentState(TypedDict):
    # Raw email data
    email_content: str
    sender_email: str
    email_id: str

    # Classification result
    classification: EmailClassification | None

    # Raw search/API results
    search_results: list[str] | None  # List of raw document chunks
    customer_history: dict | None  # Raw customer data from CRM

    # Generated content
    draft_response: str | None
    messages: list[str] | None

# Step 4: Build your nodes